<a href="https://colab.research.google.com/github/Mario-HN/Pytorch-DS/blob/main/Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
# prompt: gere uma função que sirva para usar de data_loader para o dataset CIFAR-10, ela deve preparar tanto o treino como a validação

from torchvision import datasets, transforms

def get_cifar10_data_loaders(batch_size, workers=4):
  """
  Cria os data loaders para o dataset CIFAR-10.

  Args:
    batch_size: tamanho do batch.
    workers: número de trabalhadores para carregar os dados.

  Returns:
    Uma tupla contendo os data loaders de treino e validação.
  """

  # Transformações para os dados de treino e validação.
  train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
  ])
  val_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
  ])

  # Carrega os datasets de treino e validação.
  train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=train_transform)
  val_dataset = datasets.CIFAR10(root='./data', train=False, transform=val_transform)

  # Cria os data loaders.
  train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=workers)
  val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=workers)

  return train_loader, val_loader


In [31]:
# prompt: gere uma função para usar em treinamento de redes neurais usando pytorch framework. A função deve printar a acurácia de treino e de validação a cada final de época, é interessante o uso do tqdm para iteratividade do processo. A função deve receber parâmteros de scheduler (caso necessário), learning rate, batch size, etc. podemos chamá-la de fitness function

import torch
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm

def fitness_function(model, train_loader, val_loader, scheduler=None, learning_rate=0.01, batch_size=64, epochs=10):
    """
    Fitness function for training neural networks using PyTorch.

    Args:
        model: The PyTorch model to train.
        train_loader: The DataLoader for the training set.
        val_loader: The DataLoader for the validation set.
        scheduler: The scheduler to use for learning rate decay (optional).
        learning_rate: The initial learning rate.
        batch_size: The batch size.

    Returns:
        None
    """
    # Move model to the appropriate device
    #model.to(0)

    # Define the loss function and optimizer
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, weight_decay=5e-4)

    # Train the model for the specified number of epochs
    for epoch in range(epochs):
        # Train the model for one epoch
        train_loss = 0.0
        train_correct = 0
        train_total = 0
        model.train()
        for batch in tqdm(train_loader):
            inputs, labels = batch
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            train_total += labels.size(0)
            train_correct += (predicted == labels).sum().item()

        # Evaluate the model on the validation set
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        model.eval()
        with torch.no_grad():
            for batch in val_loader:
                inputs, labels = batch
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()

        # Calculate the average losses
        train_loss /= len(train_loader)
        val_loss /= len(val_loader)

        # Print the training and validation accuracy
        train_accuracy = 100.0 * train_correct / train_total
        val_accuracy = 100.0 * val_correct / val_total
        print(f'Epoch: {epoch+1}, Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%')

        # Update the learning rate scheduler
        if scheduler:
            scheduler.step()



In [26]:
# prompt: gere uma rede de convolução bem simples para teste, ela deve receber uma entrada de 32x32x3 pixels e no final ter um flatten ligado a uma softmax para classificacao

import torch.nn as nn

class SimpleConvNet(nn.Module):
  def __init__(self):
    super(SimpleConvNet, self).__init__()

    # Camadas de convolução
    self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, padding=1)
    self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1)

    # Camada de pooling
    self.pool = nn.MaxPool2d(kernel_size=2)

    # Camada de flatten
    self.flatten = nn.Flatten()

    # Camada totalmente conectada
    self.fc = nn.Linear(in_features=32 * 8 * 8, out_features=10)

    # Camada de softmax
    self.softmax = nn.Softmax(dim=1)

  def forward(self, x):
    # Camadas de convolução e pooling
    x = self.pool(nn.ReLU()(self.conv1(x)))
    x = self.pool(nn.ReLU()(self.conv2(x)))

    # Camada de flatten
    x = self.flatten(x)

    # Camada totalmente conectada e softmax
    x = self.softmax(self.fc(x))

    return x


In [33]:
bs = 64
cnv_model = SimpleConvNet()
train_loader, val_loader = get_cifar10_data_loaders(bs)

Files already downloaded and verified


In [34]:
fitness_function(cnv_model, train_loader, val_loader, scheduler=None, learning_rate=0.01, batch_size=64, epochs=1)

100%|██████████| 782/782 [00:37<00:00, 20.59it/s]


Epoch: 1, Train Loss: 2.1751, Train Accuracy: 27.57%, Val Loss: 2.1009, Val Accuracy: 35.51%
